<a href="https://colab.research.google.com/github/gustavo-mota/ALC-Linear-Algebra/blob/main/ALC_Algoritmos_Fase01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="text-align: center">
<h1>Curso de Algebra Linear</h1>
<h3>CKP8122</h3>
<h1>Programa de Mestrado e Doutorado em Ciência da Computação - MDCC</h1>
<h3>Universidade Federal do Ceará</h3>
<h2>Gustavo Mota</h2>
<h3>2022.2</h3>
</div>


In [ ]:
import numpy as np
import cmath
from operator import itemgetter
# from scipy import linalg as scp
from scipy.sparse.linalg import cg
from scipy.linalg import svd as sp_svd
from scipy.linalg import lu as sp_lu
import sympy as sy
#from math import sqrt

In [ ]:
sample_creto = np.array([
    [6, -12, -5, 16, -2],
    [-3, 6, 3, -9, 5],
    [-4, 8, 3, -10, 1]
])

sample_cretoB = np.array([-53, 29, 33])

Algoritmo de Gauss-jordan: Pivotação parcial/eliminação e resolução

Saída esperada: vetor resposta

In [ ]:
sam = np.array([
    [1,2,3,4,5],
    [9,0,0,0,0],
    [5,5,5,5,5],
    [4,4,4,4,4],
    [0,0,0,9,9],
])

#checkNswap(sam)
sam

array([[1, 2, 3, 4, 5],
       [9, 0, 0, 0, 0],
       [5, 5, 5, 5, 5],
       [4, 4, 4, 4, 4],
       [0, 0, 0, 9, 9]])

In [ ]:
def pivota_parr(matriz, center_idx):
	n = matriz.shape[0]
	matriz = matriz.copy()

	sucess = False
	for row in range(center_idx + 1, n):

		test_value = matriz[row, center_idx]

		if not np.isclose(test_value, 0):
			matriz[center_idx, :], matriz[row, :] = matriz[row, :].copy(), matriz[center_idx, :].copy()
			sucess = True
			break

	return matriz, sucess

In [ ]:
def total_pivotation(matrix, center_idx, idx_arr):
  matrix = matrix.copy()
  n = matrix.shape[0]
  sucess = False
  
  for col in range(center_idx + 1, n):
    test_value = matrix[center_idx, col]

    if not np.isclose(test_value, 0):
      matrix[:, center_idx], matrix[:, col] = matrix[:, col].copy(), matrix[:, center_idx].copy()
      idx_arr[center_idx], idx_arr[col] = idx_arr[col], idx_arr[center_idx]

      sucess = True
      break

  return matrix, idx_arr, sucess

total_pivotation(sam, 0, np.arange(sam.shape[0]))

(array([[2, 1, 3, 4, 5],
        [0, 9, 0, 0, 0],
        [5, 5, 5, 5, 5],
        [4, 4, 4, 4, 4],
        [0, 0, 0, 9, 9]]), array([1, 0, 2, 3, 4]), True)

In [ ]:
def corr(x_b, center_idx, resp_arr_idx):

	x_b, sucess = pivota_parr(x_b, center_idx)

	if not sucess:
		x_b, resp_arr_idx, sucess = total_pivotation(x_b, center_idx, resp_arr_idx)

	if not sucess:
		print("Erro!")
		return 0, 0

	return x_b, resp_arr_idx

corr(sam, 1, np.arange(sam.shape[0]))

(array([[1, 2, 3, 4, 5],
        [5, 5, 5, 5, 5],
        [9, 0, 0, 0, 0],
        [4, 4, 4, 4, 4],
        [0, 0, 0, 9, 9]]), array([0, 1, 2, 3, 4]))

In [ ]:
def op_msr(matrix, origin_rown_idx, dest_rown_idx, shape=None, coef=1):
  origin_rown = matrix[origin_rown_idx, :]
  dest_rown = matrix[dest_rown_idx, :]
  
  matrix[dest_rown_idx, :] = dest_rown - (coef * origin_rown)
  return matrix

def divide_line(matrix, origin_rown_idx, coef=1):
  origin_rown = matrix[origin_rown_idx, :].copy()
  
  matrix[origin_rown_idx, :] = origin_rown / coef
  
  return matrix

def backward_substitution(x_b):
	n = x_b.shape[0]

	x = np.empty(n)

	x[n - 1] = x_b[n - 1, n] / x_b[n - 1, n - 1]

	for i in range(n - 2, -1, -1):
		x[i] = x_b[i, n]

		for j in range(i + 1, n):
			x[i] = x[i] - x_b[i, j] * x[j]

		x[i] = x[i] / x_b[i, i]

	return x

In [ ]:
def gauss(A, b):
  # A = A.copy()
  A = np.column_stack((A, b))
  n = A.shape[0]
  resp_order = np.arange(n)
  for i in range(n): 
    if np.isclose(A[i, i], 0):
      A, resp_order = corr(A, i, resp_order)
    
    for j in range(i + 1, n):
      alpha = A[j, i] / A[i, i]
      A = op_msr(A.copy(), origin_rown_idx=i, dest_rown_idx=j, shape=n, coef=alpha)

    A = divide_line(A.copy(), origin_rown_idx=i, coef=A[i, i])
    
  return A, resp_order

In [ ]:
sample_gausspar01 = np.array([
    [1,2], 
    [3,4]
], dtype=float)
sample_gausspar01B = np.array([1,2], dtype=float)

sample_gausspar02 = np.array([
    [1, 1, 1],
    [4, 3, 4],
    [9, 3, 4],
], dtype=float)
sample_gausspar02B = np.array([3,8,7], dtype=float)

sample_gausspar03 = np.array([
    [0.02, 0.01, 0, 0],
    [1, 2, 1, 0],
    [0, 1, 2, 1],
    [0, 0, 100, 200]
],dtype=float)
sample_gausspar03B = np.array([0.02, 1, 4, 800], dtype=float)

sample_gausspar04 = np.array([
    [5., 1., 0., 2., 1.],
    [0., 4., 0., 1., 2.],
    [1., 1., 4., 1., 1.],
    [0., 1., 2., 6., 0.],
    [0., 0., 1., 2., 4.]
],dtype=float)
sample_gausspar04B = np.array([1., 2., 3., 4., 5.], dtype=float)

# A01, B01 = eliminacao_gau(sample_gausspar01, sample_gausspar01B)
# sample_gausspar01_solu = back_substitution(A01, B01)

# A02, B02 = eliminacao_gau(sample_gausspar02, sample_gausspar02B)
# sample_gausspar02_solu = back_substitution(A02, B02)

# A03, B03 = eliminacao_gau(sample_gausspar03, sample_gausspar03B)
# sample_gausspar03_solu = back_substitution(A03, B03)

A02, B02 = gauss(sample_gausspar02.copy(), sample_gausspar02B.copy())
# sample_gausspar03_solu = back_substitution(A03, B03)

# sample_gausspar02 @ sample_gausspar02_solu
backward_substitution(A02)

array([-0.2,  4. , -0.8])

In [ ]:
A02

array([[ 1. ,  1. ,  1. ,  3. ],
       [-0. ,  1. , -0. ,  4. ],
       [-0. , -0. ,  1. , -0.8]])

In [ ]:
# np.concatenate((sample_gausspar02, sample_gausspar02B), axis=1)
#sample_gausspar02B.shape

In [ ]:
np.linalg.solve(sample_gausspar02, sample_gausspar02B)

array([-0.2,  4. , -0.8])

In [ ]:
def jordam_elim(matriz):
	matriz = matriz.copy()
	n = matriz.shape[0]

	for i in range(n-1, -1, -1):

		for j in range(i-1, -1, -1):
			alpha = matriz[j, i] / matriz[i, i]

			matriz = op_msr(matriz, origin_rown_idx=i, dest_rown_idx=j, shape=n, coef=alpha)
	return matriz


def gj_elimination(x, b):
	if(np.linalg.det(x)):

		x_forward, resp_order = gauss(x, b)

		x_bacward = jordam_elim(x_forward)

		resp = backward_substitution(x_bacward)

		return resp[resp_order]

	else: 
		print("Matriz singular!")
		return 0

In [ ]:
def switch_linhas(matriz, idx_i, idx_j): # os índices são linhas
  linha_i = np.copy(matriz[idx_i])
  linha_j = np.copy(matriz[idx_j])
  matriz[idx_i] = linha_j
  matriz[idx_j] = linha_i
  return matriz

def switch_cols(matriz, identidade, idx_i, idx_j): #os índices são colunas
  col_i, id_i = np.copy(matriz[:, idx_i]), np.copy(identidade[:, idx_i])
  col_j, id_j = np.copy(matriz[:, idx_j]), np.copy(identidade[:, idx_j])
  matriz[:, idx_i] = col_j
  matriz[:, idx_j] = col_i
  identidade[:, idx_i] = id_j
  identidade[:, idx_j] = id_i
  return matriz, identidade

def gauss_total(matriz, b):
  if(np.linalg.det(matriz)):
    output = np.copy(matriz)
    matriz_indet = np.identity(matriz.shape[0])
    idx_i, idx_j = 0, 0
    for i in range(matriz.shape[0]):
      # print("Etapa 01: pivotear o maior elemento para a posição certa")
      new_mat = matriz[idx_i:, idx_j:]
      max_value = abs(new_mat).max() # pega o maior elemento da matriz
      coords = np.argwhere(abs(matriz) == max_value)
      # print("Maior elemento de ", new_mat, " é o elemento ", max_value, " na coord: ", coords)
      # print(coords[0], idx_i, idx_j)
      if(coords[0][0] != 0):
        matriz = switch_linhas(matriz, coords[0][0], i)
        # print("Switch da iteração ", i, " matriz final: \n", matriz)
      if(coords[0][1] != 0):
        matriz, matriz_indet = switch_cols(matriz, matriz_indet, coords[0][1], i)
      # print('Matriz após reposicionamento: \n', matriz)
      # print("Etapa 2: Eliminação")
      pivot = matriz[idx_i, idx_j]
      eliminante = matriz[idx_i]*(1/pivot)
      for k in range(i+1, matriz.shape[0]):
        pivot_sub = matriz[k, i]
        #if pivot_sub < 0 and pivot < 0:
        #  eliminante = eliminante * -1
        matriz[k] = matriz[k] + eliminante*-pivot_sub
        # print("eliminação em k: ", k, " pivot sub: ", pivot_sub, " matriz[k]: ", matriz[k])
      idx_i += 1
      idx_j += 1
    
    print("Matriz final: \n", matriz, " \nidentidade: \n", matriz_indet)
    return matriz, matriz_indet
  else:
    print("Matriz singular!")
    return 0

In [ ]:
matriz_joao = np.array([[2, 1, 0, 2],
              [0, 1, 1, 3],
              [0, 2, 0, 4],
              [-4, 1, 0, 2]],
             dtype='float')
solu_joao = np.array([[5],
              [6],
              [6],
              [-1]])
mat_flavio = np.array([[2, 10, 8, 8, 6],
              [1, 4, -2, 4, -1],
              [0, 2, 3, 2, 1],
              [3, 8, 3, 10, 9],
              [1, 4, 1, 2, 1]],
             dtype='float')

solu_flavio = np.array([[52],
              [14],
              [12],
              [51],
              [15]])

# gauss_total(A, b)

#A_, b_ = eliminacao_gau(A, b)

#back_substitution(A_, b_)

In [ ]:
sample_pitotal01 = np.array([
    [17, -2, -3],
    [-5, -21, -2],
    [-5, -5, 22]
])
sample_pitotal01B = np.array([500, 200, 30])

sample_pitotal02 = np.array([
    [-2, 3, 1],
    [2, -3, -2],
    [4, 10, -6]
])

sample_pitotal03 = np.array([
    [  0.,  1.,  2.,  3.,  4.],
    [  7.,  8.,  9., 10.,  4.],
    [ 14., 15., 16., 17.,  4.],
    [  1., 20., 21., 22., 23.],
    [ 27., 28.,  1., 20., 29.]
])

sample_pitotal04 = np.array([
    [2, 4, 3],
    [1, 2, -2],
    [4, 4, 3]
])
sample_pitotal04B = np.array([1,11,3])

#gauss_total(np.copy(teste_pitotal), teste_solu_pitotal)
pitotal_out01, pitotal_id01 = gauss_total(np.copy(sample_pitotal01), sample_pitotal01B)
#pitotal_out02, pitotal_id02 = gauss_total(np.copy(sample_pitotal02), sample_pitotal02B)

Matriz final: 
 [[ 22  -5  -5]
 [  0 -21  -5]
 [  0   0  16]]  
identidade: 
 [[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [ ]:
gj_elimination(mat_flavio, solu_flavio)

# np.linalg.det(matriz_joao)

array([1., 2., 1., 2., 1.])

Gauss-Jordan

Saída esperada: matriz inversa

In [ ]:
def inter(matrix, solucao):
  matriz = matrix.copy()
  b = solucao.copy()
  linhas_dim, colunas_dim = matriz.shape
  for j in range(linhas_dim-1):
    for i in range(j+1, linhas_dim):
      alpha = (-matriz[i, j])/(matriz[j,j])
      matriz[i,j] = 0
      for k in range(j+1, colunas_dim):
        matriz[i, k] = matriz[i, k] + (alpha*matriz[j, k])
      b[i] = b[i] + (alpha*b[j])
  return (matriz, b)

def inversa_GJD(matriz, b):
  if(np.linalg.det(matriz[:, :matriz.shape[0]])):
    # print("Calcular inversa por Gauss Jordan")
    matriz, b = gauss(matriz, b)
    # print("matriz shape: ", matriz.shape)
    # print("matriz pronta: \n", matriz)
    for i in range(matriz.shape[0]-1, -1, -1):
      # print("idx i: ", i, " linha: ", matriz[i])
      pivot = matriz[i,i]
      if(pivot < 0):
        matriz[i] = matriz[i]*-1
        pivot *= -1
      for k in range(0, i):
        pivot_sub = matriz[k, i] # pega o elemento da mesma coluna do pivot
        # print("idx k: ", k, "pivot: ", pivot_sub, " linha: ", matriz[k])
        matriz[k] = matriz[k] + matriz[i]*(-pivot_sub)
    return matriz
  else:
    print("matriz singular!")
    return 0


#inv_joao = inversa_GJD(matriz_joao, solu_joao)
#matriz_joao @ inv_joao

inv_joao = inversa_GJD(mat_flavio, solu_flavio)
# matriz_joao @ inv_joao
inv_joao

array([[1., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 2.],
       [0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 2.],
       [0., 0., 0., 0., 1., 1.]])

In [ ]:
sample_inv01 = np.array([
    [1,1,3],
    [1,0,2],
    [0,1,2]
])
sample_inv01B = np.array([1,1,1])

sample_inv01gjd = inversa_GJD(np.concatenate((sample_inv01, np.identity(3)), axis=1), sample_inv01B)
sample_inv01np = np.around(np.linalg.inv(sample_inv01), 5)
print("minha:\n", sample_inv01gjd, "\npela numpy:\n", sample_inv01np)

minha:
 [[ 1.  0.  0.  2. -1. -2.]
 [-0.  1. -0.  2. -2. -1.]
 [ 0.  0.  1. -1.  1.  1.]] 
pela numpy:
 [[ 2. -1. -2.]
 [ 2. -2. -1.]
 [-1.  1.  1.]]


Algoritmo de Decomposição LU

In [ ]:
def decomp_lu02(matriz, b):
  matriz = matriz.copy()
  matriz = np.column_stack([matriz, b])

  
  n = matriz.shape[0]
  u = np.eye(n)
  l = np.eye(n)

  for k in range(n):

    if np.isclose(matriz[k, k], 0):
      print("Pivot inválido!")
      return 0,0,0
    
    u[k, k] = matriz[k, k]
    for i in range(k+1, n):
      alpha = matriz[i, k] / u[k, k]
      
      l[i, k] = alpha
      u[k, i] = matriz[k, i]
      
      matriz = op_msr(matriz, origin_rown_idx=k, dest_rown_idx=i, shape=n, coef=alpha)
      
  return l, u, matriz
  
def retro_lu(L, U, B):
  # L*Y = B
  # U*X = Y
  #print("Efetuando os cálculos de X e B")
  # Y = retro_l2(L, B)
  # X = retro_u2(U, Y)
  cont = np.column_stack([U, B])
  return backward_substitution(cont)

  #return X, Y

In [ ]:
sample_creto = np.array([[ 465., -251., -297.],
       [-251.,  136.,  160.],
       [-297.,  160.,  190.]])

In [ ]:
sample_LU01 = np.array([ 
    [2,2], 
    [4,9] 
  ])

sample_LU02 = np.array([
    [3,2,4],
    [1,1,2],
    [4,3,-2]
  ])
sample_LU02B = np.array([1,2,3])

sample_LU03 = np.array([
    [5., 1., 0., 2., 1.],
    [0., 4., 0., 1., 2.],
    [1., 1., 4., 1., 1.],
    [0., 1., 2., 6., 0.],
    [0., 0., 1., 2., 4.]
])
sample_LU03B = np.array([1,2,3,4,5])

# L01, U01 = decom_lu(sample_LU01)
# L02, U02 = decom_lu(sample_LU02)
# L03, U03 = decom_lu(sample_LU03)
L04, U04, R04 = decomp_lu02(sample_gausspar02, sample_gausspar02B)

L04 @ U04

array([[1., 1., 1.],
       [4., 3., 4.],
       [9., 3., 4.]])

In [ ]:
output_lu03 = retro_lu(L04.copy(), U04.copy(), R04[:, -1].copy())

In [ ]:
sample_gausspar02B

array([3., 8., 7.])

In [ ]:
output_lu03

array([-0.2,  4. , -0.8])

In [ ]:
sample_gausspar02 @ output_lu03

array([3., 8., 7.])

In [ ]:
pl, u = sp_lu(sample_LU03, permute_l=True) # pl, u
pl

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ],
       [0.2       , 0.2       , 1.        , 0.        , 0.        ],
       [0.        , 0.25      , 0.5       , 1.        , 0.        ],
       [0.        , 0.        , 0.25      , 0.34234234, 1.        ]])

In [ ]:
#back_substitution_v2(L03 @ U03, sample_LU03B)
#retro_lu(L=L03, U=U03, B=sample_LU03B)

NameError: ignored

In [ ]:
#np.linalg.solve(L03 @ U03, sample_LU03B)

RREF - Reduced Row Echelon Form

In [ ]:
def switch(matriz, idx_i, idx_j):
  linha_i = matriz[idx_i]
  linha_j = matriz[idx_j]
  matriz[idx_i] = linha_j
  matriz[idx_j] = linha_i
  return matriz

def first_zero(vetor):
  #print("CALCULANDO FIRST ZEROS")
  fz = -1
  for i in range(len(vetor)):
    if(vetor[i] == 0):
      fz += 1
    else:
      #print("fim do cálculo ", fz)
      return fz
  #print("Cehgamos a um output: todas as cols são zeros ", fz)
  return fz

def ajusta_pivots(matriz):
  # print("AJUSTA PIVOTS")
  #print("matriz recebida: \n", matriz)
  num_linhas = matriz.shape[0]
  first_zeros = [(first_zero(matriz[i]), i) for i in range(num_linhas)]
  #print("Antes de ordenar: ", first_zeros)
  first_zeros.sort(key=itemgetter(0))
  #print( "Depois de ordenar", first_zeros)
  new_matriz = np.zeros(matriz.shape)
  for idx, el in enumerate(first_zeros):
    # print(idx, el, new_matriz[idx], matriz[el[1]])
    new_matriz[idx] = matriz[el[1]]
  return new_matriz

def RREF(matriz):
  #rref_bool = True
  linhas_num, colunas_num = matriz.shape
  # print("shape da matriz: ", matriz.shape, linhas_num, colunas_num)
  pivot_idx = 0
  pivot_val = -1
  # primeira metade do escalonamento
  # zerar colunas abaixo dos pivots
  #while(rref_bool):
  #for i in range(0, colunas_num): # iteração sobre as colunas
  for j in range(0, linhas_num): # iteração osbre as linhas
    # print("######### ", j, " #########")
    linha_dom = matriz[j, :]
    # print("linha_dom", linha_dom)
    linha_new = linha_dom.copy()
    #coluna_atual = matriz[:, i]
    if pivot_idx < colunas_num:
      pivot_val = linha_dom[pivot_idx] # !
      # print("pivot_val: ", pivot_val, "pivot_idx", pivot_idx)

      if pivot_val != 0:
        linha_new = linha_new * (1/pivot_val)
        matriz[j, :] = linha_new
        # print("linha new origem do pivot: ", linha_new)
        for k in range(j+1, linhas_num):
          linha_sub = matriz[k, :]
          pivot_sub = linha_sub[pivot_idx]
          # print("linha sub ", linha_sub, ' idx ', k, ' pivot sub: ', pivot_sub)
          if(pivot_sub != 0): # pivot sub é dif de zero
            #x = ((pivot_sub)*-1)/(pivot_val)
            x = linha_new*-pivot_sub
            # print("x: ", x)
            #linha_sub = linha_sub + (linha_new * x)
            linha_sub = linha_sub + x
            # print("pivot: ", 1/pivot_val, " linha calculada: ", linha_sub)
            matriz[k, :] = linha_sub
      else:
        print("pivot já é zero")
    pivot_idx += 1
    pivot_val = -1
    # ajusta matriz de acordo com os pivots zerados
    matriz = ajusta_pivots(matriz)
    # print("#########################")
  
  # segunda metade do escalonamento
  #armazena a coluna do pivot e a linha correspondente
  print("SEGUNDA PARTE INICIANDO")
  print('Matriz recebida: \n', matriz)
  #first_zeros = [(-1, i) if (first_zero(matriz[i]) == colunas_num) else (first_zero(matriz[i])+1, i) for i in range(linhas_num)] 
  first_zeros = [(first_zero(matriz[i])+1, i) for i in range(linhas_num)] 
  # print("FIRST ZEROS DA SEGUNDA PARTE: ", first_zeros)
  for m in range(linhas_num-1, -1, -1):
    linha_dom = matriz[m]
    # print("linha_dom: ", linha_dom, ' m: ', m)
    coluna_pivot = first_zeros[m][0]
    if(coluna_pivot != colunas_num):
      for n in range(0, m):
        # print("n: ", n, "coluna_pivot", coluna_pivot)
        if(matriz[n, coluna_pivot] != 0):
          linha_ajuste = matriz[n]
          matriz[n] = linha_ajuste + (linha_dom * (linha_ajuste[coluna_pivot] * -1))
    else:
      print("pulando presume-se que a linha seja só de zeros")
  
  print("*************************")
  print('Matriz final: \n', matriz)
  return matriz

In [ ]:
sample_creto = np.array([
    [6, -12, -5, 16, -2],
    [-3, 6, 3, -9, 1],
    [-4, 8, 3, -10, 1]
])

sample_cretoB = np.array([-53, 29, 33])

RREF(sample_creto)

pivot já é zero
pivot já é zero
SEGUNDA PARTE INICIANDO
Matriz recebida: 
 [[ 1. -2.  0.  2.  0.]
 [ 0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.]]
pulando presume-se que a linha seja só de zeros
*************************
Matriz final: 
 [[ 1. -2.  0.  4.  0.]
 [ 0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.]]


array([[ 1., -2.,  0.,  4.,  0.],
       [ 0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [ ]:
npsample_rref01 = np.array([
    [1, 0, 1, 3], 
    [2, 3, 4, 7], 
    [-1, -3, -3, -4]
  ])
npsample_rref02 = np.array([
    [2,-2,4,-2],
    [2,1,10,7],
    [-4,4,-8,4],
    [4,-1,14,6]
])
npsample_rref03 = np.array([
    [2, 4 ,5, 3],
    [2, 3, 6, 3],
    [1, 3, 2, 2],
    [1, 5, 7, 8],
    [0, 5, 4, 9]
])
npsample_rref04 = np.array([
    [2,-2,4,-2],
    [2,1,10,7],
    [-4,4,-8,4],
    [4,-1,14,6]
])
npsample_rref05 = np.array([
    [1, 2, 1, 0],
    [-1, 0, 3, 5],
    [1, -2, 1, 1]
], dtype='float')
npsample_rref06 = np.array([
    [1, 2],
    [3, 4],
    [5, 6]
  ], dtype='float')

RREF(npsample_rref04)

pivot já é zero
pivot já é zero
SEGUNDA PARTE INICIANDO
Matriz recebida: 
 [[ 1. -1.  2. -1.]
 [ 0.  1.  2.  3.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  0.]]
pulando presume-se que a linha seja só de zeros
*************************
Matriz final: 
 [[1. 0. 4. 0.]
 [0. 1. 2. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]]


array([[1., 0., 4., 0.],
       [0., 1., 2., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.]])

In [ ]:
sysample_rref = sy.Matrix([
    [1, 0, 1, 3], 
    [2, 3, 4, 7], 
    [-1, -3, -3, -4]
  ]) # 3x4
sysample_rref02 = sy.Matrix([
    [2,-2,4,-2],
    [2,1,10,7],
    [-4,4,-8,4],
    [4,-1,14,6]
]) # 4x4
sysample_rref03 = sy.Matrix([
    [2, 4 ,5, 3],
    [2, 3, 6, 3],
    [1, 3, 2, 2],
    [1, 5, 7, 8],
    [0, 5, 4, 9]
]) # 5x4
sysample_rref04 = sy.Matrix([
    [2,-2,4,-2],
    [2,1,10,7],
    [-4,4,-8,4],
    [4,-1,14,6]
])
sysample_rref05 = sy.Matrix([
    [1, 2, 1, 0],
    [-1, 0, 3, 5],
    [1, -2, 1, 1]
])
sysample_rref06 = sy.Matrix([
    [1, 2],
    [3, 4],
    [5, 6]
])

sysample_out06 = sysample_rref06.rref()
sysample_out05 = sysample_rref05.rref()
sysample_out04 = sysample_rref04.rref()
sysample_out03 = sysample_rref03.rref()
sysample_out02 = sysample_rref02.rref()
sysample_out = sysample_rref.rref()
sysample_out04[0]

Matrix([
[1, 0, 4, 0],
[0, 1, 2, 0],
[0, 0, 0, 1],
[0, 0, 0, 0]])

Decomposição de Cholesky

In [ ]:
def define_ii(G_, matriz, ix, iy):
  print("Iniciando cálculo do elemento da diagonal: ", ix, iy)
  somatorio = 0
  for k in range(ix): # itera pelo num de cols
    print("k: ", k)
    somatorio += G_[ix, k]*G_[ix, k]
  if(somatorio > matriz[ix, iy]):
    print("Matriz não posi-definida, elemento revelado: ", matriz[ix, iy], " coords: ", ix, iy)
    return -1
  
  gii = np.sqrt(matriz[ix, iy] - somatorio)
  print("Somatorio deu: ", somatorio, " elemento da diagonal deu: ", gii, " elemento A[i,i]: ", matriz[ix,iy])
  return gii

def define_ij(G_, matriz, ix, iy):
  print("Iniciando cálculo do elemento de coord: ", ix, iy)
  print("Calculando somatório")
  somatorio = 0
  for k in range(iy):
    somatorio += G_[ix, k] * G_[iy, k]
  gij = (matriz[ix, iy] - somatorio)/G_[iy, iy]
  return gij

def define_i1(G_, matriz, g11):
  for i in range(1, matriz.shape[0]): #iterando por todas as linhas
    G_[i, 0] = matriz[i, 0]/g11
  print(G_[:, 0])
  return G_

def decompo_cholesky(matriz):
  G_ = np.zeros(matriz.shape) # criando amtriz resposta de mesmo shape
  if(matriz[0,0] < 0):
    print("Matriz não posi-definida, elemento revelado: ", matriz[0,0], " coords: ", 0, 0)
    return -1
  print("Calculando g11...")
  g11 = np.sqrt(matriz[0,0])
  G_[0,0] = g11
  print("g11 equivale a ", matriz[0,0])
  print("Definindo elementos da primeira coluna: ")
  G_ = define_i1(G_, matriz, g11)
  print("Iterando sobre os elementos da diagonal e definindo elementos das demais linhas/colunas...")
  iten = 0
  for ix, iy in np.ndindex(matriz.shape):
    if(iy == ix and iy != 0):
      iten = define_ii(G_, matriz, ix, iy)
      G_[ix, iy] = iten
    elif(iy < ix and iy != 0):
      iten = define_ij(G_, matriz, ix, iy)
      G_[ix, iy] = iten
    
    if(iten == -1):
      break
  print("Cálculo de G concluído com sucesso! Retornando")
  return G_

In [ ]:
G1 = decompo_cholesky(sample_creto)
G1

Calculando g11...
g11 equivale a  465.0
Definindo elementos da primeira coluna: 
[ 21.56385865 -11.63984628 -13.7730452 ]
Iterando sobre os elementos da diagonal e definindo elementos das demais linhas/colunas...
Iniciando cálculo do elemento da diagonal:  1 1
k:  0
Somatorio deu:  135.48602150537636  elemento da diagonal deu:  0.7169229349265116  elemento A[i,i]:  136.0
Iniciando cálculo do elemento de coord:  2 1
Calculando somatório
Iniciando cálculo do elemento da diagonal:  2 2
k:  0
k:  1
Somatorio deu:  189.8912133891214  elemento da diagonal deu:  0.329828153556682  elemento A[i,i]:  190.0
Cálculo de G concluído com sucesso! Retornando


array([[ 21.56385865,   0.        ,   0.        ],
       [-11.63984628,   0.71692293,   0.        ],
       [-13.7730452 ,  -0.4409526 ,   0.32982815]])

In [ ]:
G1 @ G1.T

array([[ 465., -251., -297.],
       [-251.,  136.,  160.],
       [-297.,  160.,  190.]])

In [ ]:
teste_cholesky = np.array([
    [4, 12, -16],
    [12, 37, -43],
    [-16, -43, 98]
])

G = decompo_cholesky(teste_cholesky)
Gt = np.transpose(G)
print(G @ Gt)

Calculando g11...
g11 equivale a  4
Definindo elementos da primeira coluna: 
[ 2.  6. -8.]
Iterando sobre os elementos da diagonal e definindo elementos das demais linhas/colunas...
Iniciando cálculo do elemento da diagonal:  1 1
k:  0
Somatorio deu:  36.0  elemento da diagonal deu:  1.0  elemento A[i,i]:  37
Iniciando cálculo do elemento de coord:  2 1
Calculando somatório
Iniciando cálculo do elemento da diagonal:  2 2
k:  0
k:  1
Somatorio deu:  89.0  elemento da diagonal deu:  3.0  elemento A[i,i]:  98
Cálculo de G concluído com sucesso! Retornando
[[  4.  12. -16.]
 [ 12.  37. -43.]
 [-16. -43.  98.]]


Ortogonalização de Gram-Schmidt

In [ ]:
def orto_gram_schimidt(matriz): # cada coluna da matriz equivale a um vetor
  print("Calculando a base ortogonal")
  conj_W = []
  W1 = matriz[:, 0]
  #conj_W = np.append(conj_W, W1)
  conj_W.append(W1)
  print("Base com primeiro elemento: ", conj_W)
  for vetor_idx in range(1, matriz.shape[1]):
    vetor = matriz[:, vetor_idx] # vetor atual para achar o seu w
    print("vetor da rodada: ", vetor)
    acumulado = vetor
    for w_idx in range(vetor_idx):
      wi = conj_W[w_idx]
      print("wi da rodada: ", wi)
      alpha = np.inner(vetor, wi)/np.inner(wi, wi)
      print("Alpha calculado: ", alpha)
      acumulado = acumulado - alpha * wi
      #conj_W = np.append(conj_W, w2)
    #w2 = vetor - acumulado
    conj_W.append(acumulado)
  return conj_W

In [ ]:
# orto_gram_schimidt()

In [ ]:
'''np.inner(
    np.array([3, 4]), np.array([2, -1])
)/np.inner(
    np.array([2, -1]), np.array([2, -1])
)'''

teste_orto = np.transpose(np.array([
    [2, -1], [3, 4]
]))

print(teste_orto[:, 0])

#conj_W = orto_gram_schimidt(teste_orto)

teste_ortor3 = np.transpose(np.array([
    [1,0,1], [1,1,0],[0,0,1]
]))

conj_W = orto_gram_schimidt(teste_ortor3)

print(conj_W)

[ 2 -1]
Calculando a base ortogonal
Base com primeiro elemento:  [array([1, 0, 1])]
vetor da rodada:  [1 1 0]
wi da rodada:  [1 0 1]
Alpha calculado:  0.5
vetor da rodada:  [0 0 1]
wi da rodada:  [1 0 1]
Alpha calculado:  0.5
wi da rodada:  [ 0.5  1.  -0.5]
Alpha calculado:  -0.3333333333333333
[array([1, 0, 1]), array([ 0.5,  1. , -0.5]), array([-0.33333333,  0.33333333,  0.33333333])]


Cálculo da ortonormalização de gram-schimidt

In [ ]:
def ortono_gram_schimidt(matriz): # calcula a ortonormal
  v1 = matriz[:, 0]
  w1 = v1/np.linalg.norm(v1) # !
  # output: lista de wi
  W_ = []
  W_.append(w1)
  # cada coluna da matriz equivale a um vetor
  for vec_idx in range(1, matriz.shape[1]):
    vi = matriz[:, vec_idx]
    print("Vetor atual da iteração apra encontrar seu w: ", vec_idx)
    somatorio = 0
    for k in range(0, vec_idx):
      wk = W_[k]
      print("wk da rodada: ", wk)
      alpha = np.inner(vi, wk)/np.inner(wk, wk) # ! # (vi*qk) <-> alpha
      print("Alpha calculado: ", alpha)
      somatorio = somatorio - alpha * wk
    normalizado = wk/np.linalg.norm(wk) # !
    W_.append(wk)
  return W_

In [ ]:
conj_W_ = ortono_gram_schimidt(np.array(conj_W))
print(conj_W_)

Vetor atual da iteração apra encontrar seu w:  1
wk da rodada:  [ 0.85714286  0.42857143 -0.28571429]
Alpha calculado:  0.33333333333333337
Vetor atual da iteração apra encontrar seu w:  2
wk da rodada:  [ 0.85714286  0.42857143 -0.28571429]
Alpha calculado:  0.5476190476190477
wk da rodada:  [ 0.85714286  0.42857143 -0.28571429]
Alpha calculado:  0.5476190476190477
[array([ 0.85714286,  0.42857143, -0.28571429]), array([ 0.85714286,  0.42857143, -0.28571429]), array([ 0.85714286,  0.42857143, -0.28571429])]


Mínimos quadrados

In [ ]:
def minimos_quads(matriz, B):
  # transforma a matriz em At*A*x = At*B
  At = np.transpose(matriz)
  AtA = np.dot(At, matriz)
  AtB = np.dot(At, B)
  x = np.linalg.solve(AtA, AtB)
  return x

In [ ]:
def test_linalg_solve():
  m = np.array([
      [6, 2],
      [2, 4]
  ])

  b = np.array([4, 4])

  x = np.linalg.solve(m, b)
  print(x)

def test_mim_quads():
  matriz = np.array([
      [-1, 1],
      [0, 1],
      [1, 1],
      [2, 1]
  ])
  B = np.array([0,1,2,1])
  out = minimos_quads(matriz, B)
  print(out)
test_mim_quads()

[0.4 0.8]
